In [1]:
# tf.assign和 = 的区别：https://www.jianshu.com/p/8e45549db258
# tf.assign(ref, value) 是将value赋值给ref ref必须是tf.Variable创建的tensor
# = python中的等号 是 贴标签 不改变tensorflow的图
# 创建了图节点的函数 每次run一下就会执行节点操作

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# 载入数据集
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# 每批次训练数据大小 和 批次大小
batch_size = 40
n_batch = mnist.train.num_examples // batch_size                                                       # 这句看不懂 一共有多少个批次 整除

# 给输入数据预留位置
x = tf.placeholder(tf.float32, shape = [None,784])
y = tf.placeholder(tf.float32, shape = [None,10])
keep_prob = tf.placeholder(tf.float32)
learning_rate = tf.Variable(0.001,dtype = tf.float32)                                                                   # 这个dtype看不懂

# 创建网络
W1 = tf.Variable(tf.truncated_normal([784,500],stddev = 0.1))                                                               # 784个2000列
b1 = tf.Variable(tf.zeros([1,500]) + 0.1)                                                                                     # 2000个1列
Wx_plus_b1 = tf.matmul(x,W1) + b1
L1 = tf.nn.tanh(Wx_plus_b1)
L1_drop = tf.nn.dropout(L1,keep_prob)

W2 = tf.Variable(tf.truncated_normal([500,300],stddev = 0.1))
b2 = tf.Variable(tf.zeros([1,300]) + 0.1)
Wx_plus_b2 = tf.matmul(L1_drop,W2) + b2
L2 = tf.nn.tanh(Wx_plus_b2)
L2_drop = tf.nn.dropout(L2,keep_prob)

W3 = tf.Variable(tf.truncated_normal([300,50],stddev = 0.1))
b3 = tf.Variable(tf.zeros([1,50]) + 0.1)
Wx_plus_b3 = tf.matmul(L2_drop,W3) + b3
L3 = tf.nn.tanh(Wx_plus_b3)
L3_drop = tf.nn.dropout(L3,keep_prob)

W4 = tf.Variable(tf.truncated_normal([50,10],stddev = 0.1))
b4 = tf.Variable(tf.zeros([1,10]) + 0.1)
Wx_plus_b4 = tf.matmul(L3_drop,W4) + b4

# 预测值
prediction = tf.nn.softmax(Wx_plus_b4)
# 交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits = Wx_plus_b4))
# 优化器
# train = tf.train.AdamOptimizer(learning_rate).minimize(loss)
train = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

# 定义求准确率的方法
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))     # 返回 Ture False 求出最大的数在的位置 看它俩返回的位置一样不
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))                                        # cast是强制类型转换 然后 求均值

# 初始化变量
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):                                                                                              # 两层循环是训练
        sess.run(tf.assign(learning_rate,0.001 * (0.99 ** epoch)))# **是幂次方
        lr = sess.run(learning_rate)
        for pic in range(n_batch):                               # 每次训练 是从第n个批次里取1个数据 循环结束 所有批次中的数据被训练一次
            pic_data,pic_tag = mnist.train.next_batch(batch_size)   # 从train集中选取batch_size个训练数据 next_batch(我觉得这是bagging 错)
            sess.run(train,feed_dict = {x:pic_data,y:pic_tag,keep_prob:1.0})                            # 获取一个训练数据 这儿就训练一次   
        # 每一次大循环 看一下准确率
        if epoch % 5 == 0:
            train_accuracy_rate = sess.run(accuracy,feed_dict = {x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})# 训练集正确率
            test_accuracy_rate = sess.run(accuracy,feed_dict = {x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})   # 测试集正确率
            print('第' + str(epoch + 1) + '次，train准确率 ' + str(train_accuracy_rate) + ' test准确率 ' + str(test_accuracy_rate)
                  + ' 学习率 ' + str(lr))
tf.test.gpu_device_name()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input

InternalError: Blas GEMM launch failed : a.shape=(40, 784), b.shape=(784, 500), m=40, n=500, k=784
	 [[node MatMul (defined at <ipython-input-3-a0154d9ef2dc>:17) ]]

Caused by op 'MatMul', defined at:
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\asyncio\base_even

In [4]:
# 第1次，train准确率 0.9620909 test准确率 0.956 学习率 0.001
# 第2次，train准确率 0.9788 test准确率 0.968 学习率 0.00099
# 第3次，train准确率 0.9776545 test准确率 0.9677 学习率 0.0009801
# 第4次，train准确率 0.9852 test准确率 0.9717 学习率 0.000970299
# 第5次，train准确率 0.98587275 test准确率 0.9705 学习率 0.000960596
# 第6次，train准确率 0.99078184 test准确率 0.9759 学习率 0.00095099007
# 第7次，train准确率 0.9921273 test准确率 0.9729 学习率 0.00094148016
# 第8次，train准确率 0.99072725 test准确率 0.9718 学习率 0.00093206536
# 第9次，train准确率 0.9913818 test准确率 0.9716 学习率 0.0009227447
# 第10次，train准确率 0.9940182 test准确率 0.9744 学习率 0.00091351726